# Projeto Final - Análise de Gols em uma Partida de Futebol
Autor: Paulo Victor Lima |   Orientador : Sergio Lima Netto |    Universidade Federal do Rio de Janeiro - Escola Politécnica - Departamente de Engenharia Eletrônica e de Computação

## Importação das Bibliotecas

In [409]:
# #Instalação das bibliotecas

# !pip install pandas
# # !pip install numpy
# # !pip install seaborn
# # !pip install matplotlib.pyplot
# # !pip install plotly.express
# # !pip install glob
# !pip install yellowbrick
# !pip install Scipy

In [410]:
#Importação das bibliotecas

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import glob
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import seaborn as sns
from numpy import loadtxt
from yellowbrick.classifier import ConfusionMatrix
from sklearn.metrics import confusion_matrix
import time
from IPython.display import clear_output
from scipy.signal import medfilt




## Importação Base de Dados

### Set Config

Classe set_config()

In [411]:
class set_config():
    def __init__(self, match_generic_path, highlights_file_name):

        #Carrega as features a serem consideradas
        features_types = []
        features_types = loadtxt("..\config\Features_Types.dat", comments="#", delimiter='\n', dtype=str, ndmin=1)
        self.features_types = features_types

        ## Caminho padrão das partidas
        self.match_generic_path = match_generic_path

        # Nome Padrão arquivo "Highlights"
        self.highlights_file_name = highlights_file_name

set_config() para treinamentos

In [412]:
def set_config_treinamento():
    
        match_generic_path = '..\\database\\Treinamento\\**\\'

        highlights_file_name = 'highlights.csv'
        
        return set_config(match_generic_path, highlights_file_name)

set_config() para testes

In [413]:
def set_config_teste():
    
        match_generic_path = '..\\database\\Teste\\**\\'

        highlights_file_name = 'highlights.csv'
        
        return set_config(match_generic_path, highlights_file_name)

set_config() para todas as partidas

In [414]:
def set_config_Aux():
    
        match_generic_path = '..\\database\\Teste_e_Treinamento\\**\\'

        highlights_file_name = 'highlights.csv'
        
        return set_config(match_generic_path, highlights_file_name)

In [415]:
def set_feature_group_paths():
    grupo1 = "..\config\Grupos\Grupo1.dat"
    grupo2 = "..\config\Grupos\Grupo2.dat"
    grupo3 = "..\config\Grupos\Grupo3.dat"
    grupo4 = "..\config\Grupos\Grupo4.dat"
    grupo5 = "..\config\Grupos\Grupo5.dat"
    grupo6 = "..\config\Grupos\Grupo6.dat"
    grupo7 = "..\config\Grupos\Grupo7.dat"
    grupo8 = "..\config\Grupos\Grupo8.dat"
    grupo9 = "..\config\Grupos\Grupo9.dat"

    feature_group_paths = [grupo1,grupo2,grupo3,grupo4,grupo5,grupo6,grupo7,grupo8,grupo9]
    # feature_group_paths = [grupo1]

    return feature_group_paths

### Leitura Base de Dados

Definição das Funções de leitura da base de dados

In [416]:
def import_moments_classifictaion(match, len, highlights_file_name):
    
    #Caminho para os arquivos de features
    match_by_feature_path = match + highlights_file_name

    #Data frame com todos os intervalos de highligths de uma partida
    highlights_file =  pd.DataFrame()

    #nome das colunas do frame de highlights
    colnames=['first_frame','last_frame', 'classification'] 

    #Data Frame com todos os highlights de uma partida
    highlights_file = pd.read_csv(match_by_feature_path, names=colnames, index_col=None, sep =',', header=None)

    #Data frame com os intervalos de highligths de uma partida
    highlights_of_match = pd.DataFrame(list(range(1,len)), columns=['classification_perigo'], index=list(range(1,len)))

    highlights_of_match['classification_perigo'] = np.nan
    highlights_of_match['classification_gol'] = highlights_of_match['classification_perigo']
    
    for index,row in highlights_file.iterrows():        
        first_frame = int(row['first_frame']) if int(row['first_frame']) != ' ' else 0
        last_frame = int(row['last_frame']) if row['last_frame'] != ' ' else int(row['first_frame'])
        classification_perigo = 'Perigo' if (row['classification'] == ' Gol' or row['classification'] == ' Perigo' or row['classification'] == ' Muito Perigo' ) else 'Normal'
        classification_gol = 'Gol' if row['classification'] == ' Gol' else 'Não Gol'


        highlights_of_match.loc[first_frame:last_frame, ['classification_perigo','classification_gol']] = [classification_perigo,classification_gol]
        
    return highlights_of_match


In [417]:
def import_database (set_config):

    from numpy import loadtxt

    #Carrega as features a serem consideradas
    features_types = []
    features_types = loadtxt("..\config\Features_Types.dat", comments="#", delimiter='\n', dtype=str, ndmin=1)

    ## Caminho padrão das partidas
    match_generic_path = set_config.match_generic_path

    #Lista de partidas existentes na base
    all_matches = glob.glob(match_generic_path, recursive = False)

    ## Data Frame com todos os momementos de todas as partidas
    all_moments = pd.DataFrame()
    ## Array de Data Frames com todos os Data Frames das partidas
    all_moments_array = []

    for match in all_matches:
        print(match)

        #Data frame com todos os momentos de uma partida
        moments_of_match=  pd.DataFrame()

        for feature in features_types:

            #Caminho para os arquivos de features
            match_by_feature_path = match + feature

            #Data Frame com todos os momentos de uma partida de uma feature especifica
            moments_of_match[feature] = pd.read_csv(match_by_feature_path, index_col=None, sep ='\n', thousands=r".", header=None)
        
        #Preenche a coluna 'Match' com a partida
        moments_of_match['Match'] = [match_by_feature_path.split('\\')[-2] for _ in range(len(moments_of_match[feature]))]

        moments_of_match[['Classification_Perigo','Classification_Gol']] = import_moments_classifictaion(match, len(moments_of_match[feature]), set_config.highlights_file_name)  
        # moments_of_match['Classification'] = import_moments_classifictaion(match, len(moments_of_match[feature]), set_config.highlights_file_name)

        moments_of_match.dropna(axis=0, how='any',inplace=True)

        #Adiciona ao array de Data Frames o Data Frame gerado para a partida
        all_moments_array.append(moments_of_match)

    all_moments = pd.concat(all_moments_array,  sort=False, axis=0, ignore_index=True)


    all_moments.to_csv( set_config.match_generic_path.split('\\')[-3] + "moments.csv", index=False, sep=';')
    return all_moments

    

## Divisão Previsores e Classe

Previsores

In [418]:
def get_predictors(all_moments, columns_names):
    return all_moments[columns_names].values

Classes

In [419]:
def get_classes_perigo(all_moments):
    return all_moments['Classification_Perigo'].values

In [420]:
def get_classes_gol(all_moments):
    return all_moments['Classification_Gol'].values

## Treinamento

In [421]:
def treinamento(N,predictorsTreinamento, classesTreinamento, predictorsTeste):
        ##Cria o Random Forest e realiza o treinamento
        random_forest = RandomForestClassifier(n_estimators=N,criterion='entropy',random_state=0, class_weight="balanced")
        random_forest.fit(predictorsTreinamento, classesTreinamento)

        return random_forest.predict(predictorsTeste)
    

## Geração dos Resultados

In [422]:
def median_filter(predictions, classificacaoPositiva, classificacaoNegativa):
    #Aplica o Filtro de Mediana
        binary_preditction =  [1 if x== classificacaoPositiva else 0 for x in predictions]

        return [classificacaoPositiva if x==1 else classificacaoNegativa for x in medfilt(binary_preditction)]
    

In [423]:
def get_Results_One_stage(df_results, N, group_name, predictors_treinamento, classes_treinamento_gol, predictors_teste, classes_teste_gol):
        start_time = time.time()
        predictions = treinamento(N, predictors_treinamento, classes_treinamento_gol, predictors_teste)
        duration_first_stage = round(time.time() - start_time, 2)

        predictions_after_filter =  median_filter(predictions, 'Gol', 'Não Gol')

        result = confusion_matrix(classes_teste_gol,predictions_after_filter).ravel()

        result = np.append(result, duration_first_stage)
        result = np.append(accuracy_score(classes_teste_gol, predictions_after_filter), result)
        result = np.append(result, N)
        result = np.append(result, group_name)
        df_result = pd.DataFrame(result).transpose()
        df_result.columns = ["Assertividade Total", "Verdadeiro Positivo","Falso Negativo","Falso Positivo","Verdadeiro Negativo","Tempo(s)", "N", "Grupo"]
        return pd.concat([df_results, df_result], axis=0, ignore_index=True)

In [424]:
def get_Second_Stage_Df(features_in_group, predictors_teste, classes_teste_perigo, predictions_perigo_after_filter, classes_teste_gol):
        df_predictor_teste = pd.DataFrame(predictors_teste)
        df_classes_teste_perigo = pd.DataFrame(classes_teste_perigo)
        df_predictions_perigo_after_filter = pd.DataFrame(np.array(predictions_perigo_after_filter))
        df_classes_teste_gol = pd.DataFrame(classes_teste_gol)

        df_second_stage_intermediary = pd.concat([df_predictor_teste, df_classes_teste_perigo, df_predictions_perigo_after_filter, df_classes_teste_gol], axis=1, ignore_index=True)

        column_names = ['Classification_Teste_Perigo', 'Prediction_Teste_Perigo', 'Classification_Gol']
        column_names = np.append(features_in_group, column_names)
        df_second_stage_intermediary.columns = column_names

        #Filtra Apenas os casos que foram classificados Corretamente no primeiro estágio
        df_second_stage_intermediary = df_second_stage_intermediary.loc[(df_second_stage_intermediary['Classification_Teste_Perigo'] == df_second_stage_intermediary['Prediction_Teste_Perigo'])]
        #Filtra Apenas os casos que foram classificados como Perigo no primeiro estágio
        return df_second_stage_intermediary.loc[(df_second_stage_intermediary['Classification_Teste_Perigo'] == 'Perigo')]

        

In [425]:
def get_Results_Two_Stages(df_results, features_in_group, N, group_name, predictors_treinamento, classes_treinamento_perigo, classes_treinamento_gol, predictors_teste, classes_teste_perigo, classes_teste_gol):
        
        #######################################Primeiro Estágio####################################
        start_time = time.time()
        predictions_perigo = treinamento(N, predictors_treinamento, classes_treinamento_perigo, predictors_teste)
        duration_first_stage = round(time.time() - start_time, 2)

        predictions_perigo_after_filter =  median_filter(predictions_perigo, 'Perigo', 'Normal')
    
        result_first_stage = confusion_matrix(classes_teste_perigo, predictions_perigo_after_filter).ravel()
        accuracy_first_stage = accuracy_score(classes_teste_perigo, predictions_perigo_after_filter)

        result = np.append(accuracy_first_stage, result_first_stage)
        result = np.append(result, duration_first_stage)


        #######################################Segundo Estágio####################################
        df_second_stage = get_Second_Stage_Df(features_in_group, predictors_teste, classes_teste_perigo, predictions_perigo_after_filter, classes_teste_gol)

        # Separa Preditores e Classificadores da base de teste
        predictors_teste_second_stage = get_predictors(df_second_stage,features_in_group)
        classes_teste_gol_second_stage = get_classes_gol(df_second_stage)

        start_time = time.time()
        predictions_gol = treinamento(N, predictors_treinamento, classes_treinamento_gol, predictors_teste_second_stage)

        duration_second_stage = round(time.time() - start_time, 2)

        predictions_gol_after_filter =  median_filter(predictions_gol, 'Gol', 'Não Gol')
        
        accuracy_second_stage = accuracy_score(classes_teste_gol_second_stage, predictions_gol_after_filter)
        result_second_stage = confusion_matrix(classes_teste_gol_second_stage, predictions_gol_after_filter).ravel()

        result = np.append(result, accuracy_second_stage)
        result = np.append(result, result_second_stage)
        result = np.append(result, duration_second_stage)

        ######################################Resultados Dois Estágios ##########################

        result = np.append(result, N)
        result = np.append(result, group_name)
        df_result = pd.DataFrame(result).transpose()
        column_names = ["Assertividade Total - Primeiro Estagio", "Verdadeiro Negativo - Primeiro Estagio","Falso Positivo - Primeiro Estagio", "Falso Negativo - Primeiro Estagio", "Verdadeiro Positivo - Primeiro Estagio","Tempo(s) - Primeiro Estagio","Assertividade Total - Segundo Estagio", "Verdadeiro Positivo - Segundo Estagio","Falso Negativo - Segundo Estagio","Falso Positivo - Segundo Estagio","Verdadeiro Negativo - Segundo Estagio","Tempo(s) - Segundo Estagio", "N", "Grupo"]
        df_result.columns = column_names
        return pd.concat([df_results, df_result], axis=0, ignore_index=True)

In [426]:
def get_Results(features_in_group, group_name, moments_treinamento, moments_test):
    ############################################################################################

    # Separa Preditores e Classificadores da base de treinamento
    predictors_treinamento = get_predictors(moments_treinamento, features_in_group)
    classes_treinamento_perigo = get_classes_perigo(moments_treinamento)
    classes_treinamento_gol = get_classes_gol(moments_treinamento)

    # Separa Preditores e Classificadores da base de teste
    predictors_teste = get_predictors(moments_test,features_in_group)
    classes_teste_perigo = get_classes_perigo(moments_test)
    classes_teste_gol = get_classes_gol(moments_test)

    # print('Primeiro Estágio')
    # Verifica_Importancia(predictors_treinamento, classes_treinamento_perigo)
    # print('Segundo Estágio')
    # Verifica_Importancia(predictors_treinamento, classes_treinamento_gol)

 

    df_results_one_stage = pd.DataFrame()
    df_results_two_stages = pd.DataFrame()
    for N in range(1,101,3):
    # for N in range(1,2):


        print('Valor de N: %d ' % (N))
        df_results_one_stage = get_Results_One_stage(df_results_one_stage, N, group_name, predictors_treinamento, classes_treinamento_gol, predictors_teste, classes_teste_gol)
        # df_results_two_stages = get_Results_Two_Stages(df_results_two_stages, features_in_group, N, group_name, predictors_treinamento, classes_treinamento_perigo, classes_treinamento_gol, predictors_teste, classes_teste_perigo, classes_teste_gol)


    df_results_one_stage = df_results_one_stage[["Assertividade Total", "Verdadeiro Positivo","Verdadeiro Negativo","Falso Positivo","Falso Negativo","Tempo(s)",  "N", "Grupo"]]
    # df_results_two_stages = df_results_two_stages[["Assertividade Total - Primeiro Estagio", "Verdadeiro Positivo - Primeiro Estagio","Verdadeiro Negativo - Primeiro Estagio","Falso Positivo - Primeiro Estagio","Falso Negativo - Primeiro Estagio","Tempo(s) - Primeiro Estagio","Assertividade Total - Segundo Estagio", "Verdadeiro Positivo - Segundo Estagio","Verdadeiro Negativo - Segundo Estagio","Falso Positivo - Segundo Estagio","Falso Negativo - Segundo Estagio","Tempo(s) - Segundo Estagio", "N", "Grupo"]]
    return [df_results_one_stage, df_results_two_stages]


## Função Principal

In [427]:
# Configurações Gerais de Treinamento
set_config_treinamento = set_config_treinamento()

# Configurações Gerais de Teste
set_config_teste = set_config_teste()

############################################################################################

# Importa a base de dados
all_moments_treinamento = pd.DataFrame()
all_moments_treinamento = import_database(set_config_treinamento)

# Importa a base de dados
all_moments_teste = pd.DataFrame()
all_moments_teste = import_database(set_config_teste)

###########################################################################################

# Caminhos dos Arquivos de Grupo de Atributo
feature_group_paths = set_feature_group_paths()

###########################################################################################


final_result_one_stage = pd.DataFrame()
final_result_two_stage = pd.DataFrame()
for feature_group_path in feature_group_paths:

    clear_output(wait=True)
    group_name = feature_group_path.split('\\')[-1].replace(".dat", "")
    print(group_name)
    
    features_in_group = loadtxt(feature_group_path, comments="#", delimiter='\n', dtype=str, ndmin=1)
    
####################################### GERAR CSV DE RESULTADOS de Perigo ################################

    [group_result_one_stage, group_result_two_stages] = get_Results(features_in_group, group_name, all_moments_treinamento, all_moments_teste)

    final_result_one_stage = pd.concat([final_result_one_stage, group_result_one_stage], axis=0, ignore_index=True)
    # final_result_two_stage = pd.concat([final_result_two_stage, group_result_two_stages], axis=0, ignore_index=True)


final_result_one_stage.to_csv( "..\\..\\Resultados Modelo\\RESULTADO-FINAL-UM-ESTAGIO-v2-Entropia.csv", index=False, sep=';')
# final_result_two_stage.to_csv( "..\\..\\Resultados Modelo\\RESULTADO-FINAL-DOIS-ESTAGIOS-v2-Entropia.csv", index=False, sep=';')

    

Grupo9
Valor de N: 1 
Valor de N: 4 
Valor de N: 7 
Valor de N: 10 
Valor de N: 13 
Valor de N: 16 
Valor de N: 19 
Valor de N: 22 
Valor de N: 25 
Valor de N: 28 
Valor de N: 31 
Valor de N: 34 
Valor de N: 37 
Valor de N: 40 
Valor de N: 43 
Valor de N: 46 
Valor de N: 49 
Valor de N: 52 
Valor de N: 55 
Valor de N: 58 
Valor de N: 61 
Valor de N: 64 
Valor de N: 67 
Valor de N: 70 
Valor de N: 73 
Valor de N: 76 
Valor de N: 79 
Valor de N: 82 
Valor de N: 85 
Valor de N: 88 
Valor de N: 91 
Valor de N: 94 
Valor de N: 97 
Valor de N: 100 


In [428]:
# final_result_one_stage.to_csv( "..\\..\\Resultados Modelo\\RESULTADO-FINAL-UM-ESTAGIO.csv", index=False, sep=';')
# final_result_two_stage.to_csv( "..\\..\\Resultados Modelo\\RESULTADO-FINAL-DOIS-ESTAGIOS.csv", index=False, sep=';')

## Análises Auxiliares

In [429]:
def Verifica_Importancia(predictors_treinamento, classes_treinamento):

    ##############Verificar importancia das features######################
    from cProfile import label


    random_forest = RandomForestClassifier(n_estimators=100,criterion='entropy',random_state=0)
    random_forest.fit(predictors_treinamento, classes_treinamento)

    importances = random_forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in random_forest.estimators_], axis=0)

    forest_importances = pd.Series(importances, index= ['Atributo 1','Atributo 2','Atributo 3','Atributo 4','Atributo 5','Atributo 6','Atributo 7','Atributo 8','Atributo 9','Atributo 10','Atributo 11','Atributo 12','Atributo 13','Atributo 14','Atributo 15','Atributo 16','Atributo 17','Atributo 18','Atributo 19'])


    fig, ax = plt.subplots()
    forest_importances.plot.bar(yerr=std, ax=ax)
    ax.set_title("Importâncias dos Atributos usando MDI")
    ax.set_ylabel("Redução Média de Impureza (MDI)")
    fig.tight_layout()

In [430]:
# def Verifica_Importancia(predictors_treinamento, classes_treinamento_perigo, classes_treinamento_gol):

#     ##############Verificar importancia das features Primeiro Estágio ######################
#     from cProfile import label


#     random_forest_first_stage = RandomForestClassifier(n_estimators=100,criterion='gini',random_state=0)
#     random_forest_first_stage.fit(predictors_treinamento, classes_treinamento_perigo)

#     importances_first_stage = random_forest_first_stage.feature_importances_
#     std_first_stage = np.std([tree.feature_importances_ for tree in random_forest_first_stage.estimators_], axis=0)

#     forest_importances_first_stage = pd.Series(importances_first_stage, index= ['Atributo 1','Atributo 2','Atributo 3','Atributo 4','Atributo 5','Atributo 6','Atributo 7','Atributo 8','Atributo 9','Atributo 10','Atributo 11','Atributo 12','Atributo 13','Atributo 14','Atributo 15','Atributo 16','Atributo 17','Atributo 18','Atributo 19'])


#     fig, ax = plt.subplots()
#     forest_importances_first_stage.plot.bar(yerr=std_first_stage, ax=ax)
#     ax.set_title("Importâncias dos Atributos Primeiro Estágio usando MDI")
#     ax.set_ylabel("Redução Média de Impureza (MDI)")
#     fig.tight_layout()


#     ##############Verificar importancia das features Segundo Estágio ######################
#     from cProfile import label


#     random_forest_second_stage = RandomForestClassifier(n_estimators=100,criterion='gini',random_state=0)
#     random_forest_second_stage.fit(predictors_treinamento, classes_treinamento_gol)

#     importances_second_stage = random_forest_second_stage.feature_importances_
#     std_second_stage = np.std([tree.feature_importances_ for tree in random_forest_second_stage.estimators_], axis=0)

#     forest_importances_second_stage = pd.Series(importances_second_stage, index= ['Atributo 1','Atributo 2','Atributo 3','Atributo 4','Atributo 5','Atributo 6','Atributo 7','Atributo 8','Atributo 9','Atributo 10','Atributo 11','Atributo 12','Atributo 13','Atributo 14','Atributo 15','Atributo 16','Atributo 17','Atributo 18','Atributo 19'])


#     fig, ax = plt.subplots()
#     forest_importances_second_stage.plot.bar(yerr=std_second_stage, ax=ax)
#     ax.set_title("Importâncias dos Atributos Segundo Estágio usando MDI")
#     ax.set_ylabel("Redução Média de Impureza (MDI)")
#     fig.tight_layout()

In [431]:
# # Configurações Gerais de Treinamento
# set_config_treinamento = set_config_Aux()

# all_moments = import_database(set_config_treinamento)
# sns.set(rc={'figure.figsize':(10.2,8.27)})
# fig = sns.countplot(y="Classification", hue="Match", data = all_moments, palette="tab10")

# fig.set_xlabel("Quantidade")
# fig.set_ylabel("Classificação")

# plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)


In [432]:
# sns.set(rc={'figure.figsize':(11.7,8.27)})
# fig = sns.countplot(y="Classification", data = all_moments)
# fig.set_xlabel("Quantidade")
# fig.set_ylabel("Classificação")